<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/08_RAG_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Técnicas Avanzadas de RAG para Búsqueda y Recomendación de Películas

#0. Importamos las librerias necesarias

In [31]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Tuple
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from datetime import datetime
import re

In [18]:
print("Descargando recursos de NLTK...")
import nltk

# Descarga de recursos necesarios
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
print("Recursos NLTK descargados correctamente.")


Descargando recursos de NLTK...
Recursos NLTK descargados correctamente.


## 1. Búsqueda Semántica Mejorada
- Combina sinopsis y palabras clave para mejorar la relevancia
- Usa pesos diferentes para cada componente
- Incluye extracción automática de keywords
- Demuestra cómo enriquecer los resultados con información de género




In [37]:
class SemanticSearchEnhancer:
    """
    Mejora la búsqueda semántica combinando sinopsis y palabras clave.
    """
    def __init__(self, model_name: str = 'distiluse-base-multilingual-cased-v1'):
        self.model = SentenceTransformer(model_name)
        self.synopsis_embeddings = None
        self.keyword_embeddings = None

    def extract_keywords(self, text: str) -> str:
        """Extrae palabras clave del texto."""
        # Tokenización y eliminación de stopwords
        stop_words = set(stopwords.words('spanish'))
        tokens = word_tokenize(text.lower())
        keywords = [word for word in tokens if word not in stop_words]
        return ' '.join(keywords)

    def prepare_data(self, df: pd.DataFrame):
        """Prepara los datos generando embeddings de sinopsis y keywords."""
        # Combinar sinopsis con géneros
        enhanced_texts = [
            f"{row['descripcion']} {row['genero']}"
            for _, row in df.iterrows()
        ]

        # Extraer y embeber keywords
        keywords = [self.extract_keywords(text) for text in enhanced_texts]

        # Generar embeddings
        print("Generando embeddings de sinopsis...")
        self.synopsis_embeddings = self.model.encode(enhanced_texts)
        print("Generando embeddings de keywords...")
        self.keyword_embeddings = self.model.encode(keywords)

        return self

    def search(self, query: str, df: pd.DataFrame, top_k: int = 3,
              synopsis_weight: float = 0.7) -> List[Dict]:
        """
        Realiza búsqueda semántica mejorada combinando similitud de
        sinopsis y keywords.
        """
        # Generar embedding de la query
        query_embedding = self.model.encode([query])

        # Calcular similitudes
        synopsis_scores = cosine_similarity(query_embedding,
                                         self.synopsis_embeddings)[0]
        keyword_scores = cosine_similarity(query_embedding,
                                        self.keyword_embeddings)[0]

        # Combinar scores
        combined_scores = (synopsis_scores * synopsis_weight +
                         keyword_scores * (1 - synopsis_weight))

        # Obtener top_k resultados
        top_indices = np.argsort(combined_scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append({
                'titulo': df.iloc[idx]['titulo'],
                'descripcion': df.iloc[idx]['descripcion'],
                'genero': df.iloc[idx]['genero'],
                'año': df.iloc[idx]['año'],
                'valoracion': df.iloc[idx]['valoracion'],
                'popularidad': df.iloc[idx]['popularidad'],
                'score': combined_scores[idx]
            })

        return results

## 2. Re-Ranking Contextual
- Reordena resultados basándose en el contexto actual
     - Valoraciones de usuarios (rating_score)
     - Actualidad de la película (recency_score)
     - Popularidad general (popularity_score)
     - Score de relevancia semántica original
- Usa un factor alpha para balancear scores originales y contextuales
     - Relevancia semántica (30%)
     - Actualidad (20%)
     - Valoraciones (30%)
     - Popularidad (20%)

In [38]:
class ContextualReranker:
    """
    Re-ranking de resultados basado en factores contextuales como
    valoraciones, actualidad y popularidad.
    """
    def __init__(self):
        self.current_year = datetime.now().year

    def calculate_recency_score(self, year: int) -> float:
        """
        Calcula un score basado en la actualidad de la película.
        Películas más recientes obtienen scores más altos.
        """
        age = self.current_year - year
        return 1 / (1 + 0.1 * age)  # función de decaimiento suave

    def calculate_rating_score(self, rating: float,
                             min_rating: float = 0,
                             max_rating: float = 10) -> float:
        """
        Normaliza la valoración al rango [0,1].
        """
        return (rating - min_rating) / (max_rating - min_rating)

    def calculate_popularity_score(self, popularity: float,
                                 max_popularity: float) -> float:
        """
        Normaliza el score de popularidad.
        """
        return popularity / max_popularity

    def rerank(self, results: List[Dict],
              weights: Dict[str, float] = None) -> List[Dict]:
        """
        Re-rankea resultados basándose en múltiples factores contextuales.

        Parameters:
        -----------
        results : List[Dict]
            Lista de resultados originales
        weights : Dict[str, float]
            Pesos para cada factor:
            - 'relevance': peso para el score de relevancia original
            - 'recency': peso para la actualidad
            - 'rating': peso para las valoraciones
            - 'popularity': peso para la popularidad
        """
        if weights is None:
            weights = {
                'relevance': 0.3,   # score original de relevancia
                'recency': 0.2,     # actualidad de la película
                'rating': 0.3,      # valoraciones de usuarios
                'popularity': 0.2    # popularidad general
            }

        # Encontrar máximo de popularidad para normalización
        max_popularity = max(r['popularidad'] for r in results)

        # Calcular scores contextuales
        for result in results:
            # Score de actualidad
            recency_score = self.calculate_recency_score(result['año'])

            # Score de valoración
            rating_score = self.calculate_rating_score(result['valoracion'])

            # Score de popularidad
            popularity_score = self.calculate_popularity_score(
                result['popularidad'],
                max_popularity
            )

            # Combinar todos los scores
            result['final_score'] = (
                weights['relevance'] * result['score'] +
                weights['recency'] * recency_score +
                weights['rating'] * rating_score +
                weights['popularity'] * popularity_score
            )

            # Guardar scores individuales para análisis
            result['score_components'] = {
                'relevance': result['score'],
                'recency': recency_score,
                'rating': rating_score,
                'popularity': popularity_score
            }

        # Reordenar resultados
        return sorted(results, key=lambda x: x['final_score'], reverse=True)

    def explain_ranking(self, result: Dict) -> str:
        """
        Genera una explicación del ranking para un resultado.
        """
        components = result['score_components']
        explanation = f"Ranking para '{result['titulo']}':\n"
        explanation += f"- Relevancia semántica: {components['relevance']:.3f}\n"
        explanation += f"- Factor de actualidad: {components['recency']:.3f}\n"
        explanation += f"- Valoración usuarios: {components['rating']:.3f}\n"
        explanation += f"- Índice popularidad: {components['popularity']:.3f}\n"
        explanation += f"Score final: {result['final_score']:.3f}"
        return explanation

## 3. Personalización Avanzada

- Mantiene perfiles de usuario con preferencias
- Actualiza perfiles basándose en interacciones
- Ajusta pesos de búsqueda según el perfil
- Considera géneros favoritos y búsquedas recientes

In [57]:
class PersonalizationEngine:
    """
    Motor de personalización que mantiene y utiliza perfiles de usuario
    para personalizar los resultados de búsqueda.

    El motor mantiene un registro de:
    - Preferencias de género
    - Películas que le han gustado al usuario
    - Historial de búsquedas recientes

    Y proporciona pesos personalizados para el re-ranking basados en este perfil.
    """
    def __init__(self):
        """
        Inicializa el motor de personalización con un diccionario vacío de perfiles.
        """
        self.user_profiles = {}

    def _initialize_profile(self, user_id: str) -> None:
        """
        Inicializa un nuevo perfil de usuario con estructura predefinida.

        Args:
            user_id (str): Identificador único del usuario
        """
        if user_id not in self.user_profiles:
            self.user_profiles[user_id] = {
                'genre_preferences': {},     # Contador de géneros preferidos
                'recent_searches': [],       # Lista de búsquedas recientes
                'liked_movies': set(),       # Conjunto de películas favoritas
                'interaction_count': 0       # Contador total de interacciones
            }

    def update_profile(self, user_id: str, interaction: Dict[str, any]) -> None:
        """
        Actualiza el perfil del usuario basado en una nueva interacción.

        Args:
            user_id (str): Identificador único del usuario
            interaction (Dict): Diccionario con la información de la interacción
                Puede contener:
                - 'genero': género(s) de la película
                - 'liked_movie': título de una película que le gustó
                - 'query': texto de búsqueda realizada
        """
        # Asegurar que existe el perfil
        self._initialize_profile(user_id)
        profile = self.user_profiles[user_id]

        # Actualizar preferencias de género
        if 'genero' in interaction:
            genres = interaction['genero'].split(', ')
            for genre in genres:
                profile['genre_preferences'][genre] = \
                    profile['genre_preferences'].get(genre, 0) + 1

        # Actualizar películas que le gustaron
        if 'liked_movie' in interaction:
            profile['liked_movies'].add(interaction['liked_movie'])

        # Actualizar búsquedas recientes
        if 'query' in interaction:
            profile['recent_searches'].append(interaction['query'])
            # Mantener solo las últimas 5 búsquedas
            profile['recent_searches'] = profile['recent_searches'][-5:]

        # Incrementar contador de interacciones
        profile['interaction_count'] += 1

    def get_user_preferences(self, user_id: str) -> Dict:
        """
        Obtiene un resumen de las preferencias del usuario.

        Args:
            user_id (str): Identificador único del usuario

        Returns:
            Dict: Resumen de preferencias incluyendo géneros favoritos,
                  películas que le gustaron y búsquedas recientes
        """
        if user_id not in self.user_profiles:
            return None

        profile = self.user_profiles[user_id]
        return {
            'favorite_genres': dict(sorted(
                profile['genre_preferences'].items(),
                key=lambda x: x[1],
                reverse=True
            )),
            'liked_movies': list(profile['liked_movies']),
            'recent_searches': profile['recent_searches']
        }

    def get_personalized_weights(self, user_id: str) -> Dict[str, float]:
        """
        Calcula y devuelve pesos personalizados para el re-ranking basados
        en el perfil del usuario.

        Los pesos se ajustan según:
        - La diversidad de géneros que le gustan al usuario
        - La cantidad de interacciones realizadas
        - Sus preferencias específicas

        Args:
            user_id (str): Identificador único del usuario

        Returns:
            Dict[str, float]: Diccionario con los pesos para cada factor:
                - 'relevance': peso para relevancia semántica y género
                - 'recency': peso para la actualidad de la película
                - 'rating': peso para las valoraciones de usuarios
                - 'popularity': peso para la popularidad general
        """
        # Si no existe el usuario, devolver pesos por defecto
        if user_id not in self.user_profiles:
            return {
                'relevance': 0.3,  # combina relevancia semántica y género
                'recency': 0.2,    # actualidad de la película
                'rating': 0.3,     # valoraciones de usuarios
                'popularity': 0.2   # popularidad general
            }

        profile = self.user_profiles[user_id]

        # Analizar diversidad de géneros
        genre_diversity = len(profile['genre_preferences'])

        # Analizar nivel de interacción
        interaction_level = min(profile['interaction_count'] / 10.0, 1.0)

        if genre_diversity < 3:
            # Usuario con preferencias específicas
            return {
                'relevance': 0.4,  # más peso a relevancia y género
                'recency': 0.2,    # peso moderado a actualidad
                'rating': 0.3,     # peso significativo a valoraciones
                'popularity': 0.1   # menos peso a popularidad general
            }
        else:
            # Usuario con gustos más variados
            return {
                'relevance': 0.2,  # menos peso a relevancia específica
                'recency': 0.2,    # mantener peso de actualidad
                'rating': 0.3,     # mantener peso de valoraciones
                'popularity': 0.3   # más peso a popularidad general
            }

    def explain_weights(self, user_id: str) -> str:
        """
        Genera una explicación en lenguaje natural de los pesos asignados.

        Args:
            user_id (str): Identificador único del usuario

        Returns:
            str: Explicación detallada de los pesos y su razón
        """
        if user_id not in self.user_profiles:
            return "Usuario nuevo: usando pesos por defecto balanceados"

        profile = self.user_profiles[user_id]
        genre_diversity = len(profile['genre_preferences'])

        explanation = [
            f"Perfil del usuario:",
            f"- Ha interactuado con {profile['interaction_count']} películas",
            f"- Tiene preferencias en {genre_diversity} géneros diferentes",
            f"- Géneros favoritos: {', '.join(sorted(profile['genre_preferences'].keys()))}",
            "",
            "Estrategia de personalización:"
        ]

        if genre_diversity < 3:
            explanation.append(
                "- Mayor peso a relevancia por preferencias específicas"
            )
        else:
            explanation.append(
                "- Mayor peso a popularidad por gustos variados"
            )

        return "\n".join(explanation)

## 4. Fusión Inteligente de Contextos
- Genera consultas alternativas automáticamente    
- Combina resultados de múltiples fuentes    
- Usa pesos dinámicos para la fusión   
- Demuestra cómo mejorar la diversidad de resultados   

In [68]:
class SmartContextFusion:
    """
    Fusión inteligente de resultados de búsqueda usando múltiples
    consultas alternativas y pesos dinámicos.
    """
    def __init__(self, model_name: str = 'distiluse-base-multilingual-cased-v1'):
        self.model = SentenceTransformer(model_name)

    def generate_alternative_queries(self, query: str) -> List[str]:
        """
        Genera consultas alternativas basadas en la consulta original.

        Las consultas generadas cubren diferentes aspectos:
        - Consulta original (exactitud)
        - Similitud (películas parecidas)
        - Calidad (mejores películas)
        - Popularidad (películas populares)
        - Recomendación (películas recomendadas)

        Args:
            query (str): Consulta original del usuario

        Returns:
            List[str]: Lista de consultas alternativas
        """
        alternatives = [
            query,  # consulta original
            f"{query} similares",  # enfoque en similitud
            f"mejores {query}",  # enfoque en calidad
            f"{query} populares",  # enfoque en popularidad
            f"{query} más recomendadas"  # enfoque en recomendación
        ]
        return alternatives

    def get_default_weights(self, num_queries: int) -> List[float]:
        """
        Genera pesos por defecto para las consultas alternativas.

        Args:
            num_queries (int): Número de consultas alternativas

        Returns:
            List[float]: Lista de pesos normalizados
        """
        if num_queries <= 1:
            return [1.0]

        # Dar más peso a la consulta original
        weights = [0.3]  # consulta original

        # Distribuir el resto del peso entre las alternativas
        remaining_weight = 0.7
        weight_per_query = remaining_weight / (num_queries - 1)
        weights.extend([weight_per_query] * (num_queries - 1))

        return weights

    def fuse_results(self, all_results: List[List[Dict]],
                    weights: List[float] = None) -> List[Dict]:
        """
        Fusiona resultados de diferentes consultas usando pesos específicos.

        Args:
            all_results: Lista de listas de resultados por cada consulta
            weights: Lista de pesos para cada conjunto de resultados

        Returns:
            List[Dict]: Lista de resultados fusionados y ordenados
        """
        if not all_results:
            return []

        # Usar pesos por defecto si no se proporcionan
        if weights is None:
            weights = self.get_default_weights(len(all_results))

        # Normalizar pesos
        weights = np.array(weights) / sum(weights)

        # Combinar todos los resultados
        combined_scores = {}
        for results, weight in zip(all_results, weights):
            for result in results:
                titulo = result['titulo']
                if titulo not in combined_scores:
                    combined_scores[titulo] = {
                        'score': 0,
                        'data': result,
                        'found_in_queries': 0,
                        'max_individual_score': 0
                    }

                # Actualizar información del resultado
                combined_scores[titulo]['score'] += result['score'] * weight
                combined_scores[titulo]['found_in_queries'] += 1
                combined_scores[titulo]['max_individual_score'] = max(
                    combined_scores[titulo]['max_individual_score'],
                    result['score']
                )

        # Aplicar bonus por aparición en múltiples consultas
        for info in combined_scores.values():
            query_diversity_bonus = info['found_in_queries'] / len(all_results)
            info['score'] *= (1 + 0.2 * query_diversity_bonus)  # 20% bonus máximo

        # Ordenar por score combinado
        sorted_results = sorted(
            combined_scores.values(),
            key=lambda x: x['score'],
            reverse=True
        )

        return [item['data'] for item in sorted_results]

    def explain_fusion(self, titulo: str, combined_scores: Dict) -> str:
        """
        Genera una explicación de por qué un resultado obtuvo su score final.

        Args:
            titulo (str): Título de la película
            combined_scores (Dict): Diccionario con los scores combinados

        Returns:
            str: Explicación detallada del ranking
        """
        if titulo not in combined_scores:
            return f"No se encontró información para '{titulo}'"

        info = combined_scores[titulo]
        explanation = [
            f"Explicación del ranking para '{titulo}':",
            f"- Encontrada en {info['found_in_queries']} consultas diferentes",
            f"- Mejor score individual: {info['max_individual_score']:.3f}",
            f"- Score final combinado: {info['score']:.3f}"
        ]

        return "\n".join(explanation)

## 5. EJEMPLO DE USO

### Datos de Ejemplo

Simulamos datos de películas:

In [39]:
movies_data = {
    'titulo': [
        'El Padrino',
        'Matrix',
        'Inception',
        'La La Land',
        'Get Out',
        'Interestelar',
        'El Señor de los Anillos',
        'Parásitos',
        'Whiplash',
        'Coco',
        'Mad Max: Fury Road',
        'El Gran Hotel Budapest',
        'Black Panther',
        'Wonder Woman',
        'Arrival',
        'Ex Machina',
        'Your Name',
        'El Laberinto del Fauno',
        'Ciudad de Dios',
        'Amelie'
    ],
    'descripcion': [
        'Una familia mafiosa en Nueva York lucha por mantener su imperio criminal',
        'Un programador descubre que la realidad es una simulación computarizada',
        'Un ladrón especializado se infiltra en los sueños de sus objetivos',
        'Una aspirante a actriz y un músico de jazz persiguen sus sueños en Los Ángeles',
        'Un joven afroamericano visita a la familia de su novia con inquietantes consecuencias',
        'Un grupo de astronautas busca un nuevo hogar para la humanidad',
        'Un hobbit debe destruir un anillo mágico para salvar la Tierra Media',
        'Una familia pobre se infiltra en la vida de una familia rica',
        'Un joven baterista persigue la perfección bajo un instructor implacable',
        'Un niño viaja al mundo de los muertos durante el Día de los Muertos',
        'En un mundo post-apocalíptico, una guerrera lidera una rebelión',
        'Las aventuras de un legendario conserje de hotel en la Europa de entreguerras',
        'El príncipe de Wakanda debe defender su reino y su legado',
        'Una princesa amazona se convierte en una poderosa superheroína',
        'Una lingüista intenta comunicarse con alienígenas recién llegados',
        'Un programador participa en un experimento de inteligencia artificial',
        'Dos adolescentes japoneses intercambian cuerpos misteriosamente',
        'Una niña descubre un mundo mágico durante la posguerra española',
        'El crecimiento del crimen organizado en las favelas de Río',
        'Una camarera parisina decide ayudar a mejorar la vida de otros'
    ],
    'genero': [
        'Drama, Crimen',
        'Sci-Fi, Acción',
        'Sci-Fi, Thriller',
        'Musical, Romance',
        'Terror, Thriller',
        'Sci-Fi, Drama',
        'Fantasía, Aventura',
        'Drama, Comedia',
        'Drama, Música',
        'Animación, Fantasía',
        'Acción, Aventura',
        'Comedia, Drama',
        'Acción, Aventura',
        'Acción, Fantasía',
        'Sci-Fi, Drama',
        'Sci-Fi, Drama',
        'Animación, Romance',
        'Fantasía, Drama',
        'Drama, Crimen',
        'Comedia, Romance'
    ],
    'año': [1972, 1999, 2010, 2016, 2017, 2014, 2001, 2019, 2014, 2017, 2015, 2014, 2018, 2017, 2016, 2015, 2016, 2006, 2002, 2001],
    'valoracion': [9.2, 8.7, 8.8, 8.5, 7.7, 8.6, 8.8, 8.6, 8.5, 8.4, 8.1, 8.1, 7.3, 7.4, 7.9, 7.7, 8.4, 8.2, 8.6, 8.3],
    'popularidad': [100, 95, 90, 85, 80, 88, 98, 87, 82, 89, 86, 83, 92, 88, 84, 81, 85, 84, 87, 86]  # índice de popularidad
}
df = pd.DataFrame(movies_data)

Simulamos un perfil de usuario:

In [46]:
# Simulamos un perfil de usuario con preferencias
user_id = "user123"
personalizer = PersonalizationEngine()

# Añadimos algunas interacciones previas del usuario
interacciones_previas = [
    {'genero': 'Sci-Fi, Drama', 'liked_movie': 'Interestelar'},
    {'genero': 'Sci-Fi, Acción', 'liked_movie': 'Matrix'},
    {'query': 'películas de ciencia ficción con buena trama'},
    {'genero': 'Drama', 'liked_movie': 'Whiplash'}
]

for interaccion in interacciones_previas:
    personalizer.update_profile(user_id, interaccion)

# Obtenemos los pesos personalizados basados en el perfil
weights = personalizer.get_personalized_weights(user_id)
print("\nPerfil del usuario:")
print(f"Géneros preferidos: {dict(sorted(personalizer.user_profiles[user_id]['genre_preferences'].items(), key=lambda x: x[1], reverse=True))}")



Perfil del usuario:
Géneros preferidos: {'Sci-Fi': 2, 'Drama': 2, 'Acción': 1}


###5.1 Búsqueda Semántica Mejorada

In [65]:
import json
query = "películas de ciencia ficción"
print("\n1. Probando Búsqueda Semántica Mejorada...")
enhancer = SemanticSearchEnhancer()
enhancer.prepare_data(df)
semantic_results = enhancer.search(query, df)
for idx, result in enumerate(semantic_results):
  print(idx,result)


1. Probando Búsqueda Semántica Mejorada...
Generando embeddings de sinopsis...
Generando embeddings de keywords...
0 {'titulo': 'Inception', 'descripcion': 'Un ladrón especializado se infiltra en los sueños de sus objetivos', 'genero': 'Sci-Fi, Thriller', 'año': 2010, 'valoracion': 8.8, 'popularidad': 90, 'score': 0.4399857}
1 {'titulo': 'Interestelar', 'descripcion': 'Un grupo de astronautas busca un nuevo hogar para la humanidad', 'genero': 'Sci-Fi, Drama', 'año': 2014, 'valoracion': 8.6, 'popularidad': 88, 'score': 0.43668208}
2 {'titulo': 'Arrival', 'descripcion': 'Una lingüista intenta comunicarse con alienígenas recién llegados', 'genero': 'Sci-Fi, Drama', 'año': 2016, 'valoracion': 7.9, 'popularidad': 84, 'score': 0.43103325}


###5.2 Re-ranking Contextual

In [58]:
print("\n2. Probando Re-ranking Contextual...")
reranker = ContextualReranker()

# Definir pesos personalizados para el re-ranking
contextual_weights = {
    'relevance': 0.1,
    'recency': 0.1,
    'rating': 0.4,
    'popularity': 0.4
}

reranked_results = reranker.rerank(semantic_results, contextual_weights)

# Mostrar explicaciones del ranking
for result in reranked_results:
    print("\n" + reranker.explain_ranking(result))


2. Probando Re-ranking Contextual...

Ranking para 'Inception':
- Relevancia semántica: 0.440
- Factor de actualidad: 0.417
- Valoración usuarios: 0.880
- Índice popularidad: 1.000
Score final: 0.838

Ranking para 'Interestelar':
- Relevancia semántica: 0.437
- Factor de actualidad: 0.500
- Valoración usuarios: 0.860
- Índice popularidad: 0.978
Score final: 0.829

Ranking para 'Arrival':
- Relevancia semántica: 0.431
- Factor de actualidad: 0.556
- Valoración usuarios: 0.790
- Índice popularidad: 0.933
Score final: 0.788


###5.3 Personalización

In [62]:
# Simulamos un perfil de usuario con preferencias
personalizer = PersonalizationEngine()
user_id = "user123"

# Simular historial del usuario
historial_usuario = [
    {'genero': 'Sci-Fi, Drama', 'liked_movie': 'Interestelar'},
    {'genero': 'Sci-Fi, Acción', 'liked_movie': 'Matrix'},
    {'query': 'mejores películas de ciencia ficción'},
    {'genero': 'Drama', 'liked_movie': 'Whiplash'},
    {'query': 'películas con efectos especiales'}
]

# Actualizar perfil con el historial
for interaccion in historial_usuario:
    personalizer.update_profile(user_id, interaccion)

# Mostrar preferencias actuales
print("\nPerfil del usuario:")
preferencias = personalizer.get_user_preferences(user_id)
print(f"- Géneros favoritos: {dict(preferencias['favorite_genres'])}")
print(f"- Películas que le gustaron: {preferencias['liked_movies']}")
print(f"- Búsquedas recientes: {preferencias['recent_searches']}")

print("\n2. PESOS PERSONALIZADOS")
print("=======================")
user_weights = personalizer.get_personalized_weights(user_id)
print("\nPesos calculados según el perfil:")
for factor, peso in user_weights.items():
    print(f"- {factor}: {peso:.2f}")


print("\n3. Probando Personalización...")
reranked_results = reranker.rerank(semantic_results, user_weights)

# Mostrar explicaciones del ranking
for result in reranked_results:
    print("\n" + reranker.explain_ranking(result))


Perfil del usuario:
- Géneros favoritos: {'Sci-Fi': 2, 'Drama': 2, 'Acción': 1}
- Películas que le gustaron: ['Matrix', 'Whiplash', 'Interestelar']
- Búsquedas recientes: ['mejores películas de ciencia ficción', 'películas con efectos especiales']

2. PESOS PERSONALIZADOS

Pesos calculados según el perfil:
- relevance: 0.20
- recency: 0.20
- rating: 0.30
- popularity: 0.30

3. Probando Personalización...

Ranking para 'Interestelar':
- Relevancia semántica: 0.437
- Factor de actualidad: 0.500
- Valoración usuarios: 0.860
- Índice popularidad: 0.978
Score final: 0.739

Ranking para 'Inception':
- Relevancia semántica: 0.440
- Factor de actualidad: 0.417
- Valoración usuarios: 0.880
- Índice popularidad: 1.000
Score final: 0.735

Ranking para 'Arrival':
- Relevancia semántica: 0.431
- Factor de actualidad: 0.556
- Valoración usuarios: 0.790
- Índice popularidad: 0.933
Score final: 0.714


### 5.4. Fusión de Contextos

In [71]:
print("\n4. Probando Fusión de Contextos...")
fusion = SmartContextFusion()

print(f"\nConsulta original: '{query}'")

alt_queries = fusion.generate_alternative_queries(query)
for query_generada in alt_queries[1:]:
    print(f"Consulta alternativa: '{query_generada}'")
all_results = [enhancer.search(q, df) for q in alt_queries]
final_results = fusion.fuse_results(all_results)
for idx,result in enumerate(final_results):
  print(idx, result)


4. Probando Fusión de Contextos...

Consulta original: 'películas de ciencia ficción'
Consulta alternativa: 'películas de ciencia ficción similares'
Consulta alternativa: 'mejores películas de ciencia ficción'
Consulta alternativa: 'películas de ciencia ficción populares'
Consulta alternativa: 'películas de ciencia ficción más recomendadas'
0 {'titulo': 'Inception', 'descripcion': 'Un ladrón especializado se infiltra en los sueños de sus objetivos', 'genero': 'Sci-Fi, Thriller', 'año': 2010, 'valoracion': 8.8, 'popularidad': 90, 'score': 0.4399857}
1 {'titulo': 'Interestelar', 'descripcion': 'Un grupo de astronautas busca un nuevo hogar para la humanidad', 'genero': 'Sci-Fi, Drama', 'año': 2014, 'valoracion': 8.6, 'popularidad': 88, 'score': 0.43668208}
2 {'titulo': 'Arrival', 'descripcion': 'Una lingüista intenta comunicarse con alienígenas recién llegados', 'genero': 'Sci-Fi, Drama', 'año': 2016, 'valoracion': 7.9, 'popularidad': 84, 'score': 0.43103325}
